In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv('results.csv')

nn_df = df[df['name'] == 'nn'].reset_index()
gc_df = df[df['name'] == 'gc'].reset_index()
ls_random_df = df[df['name'] == 'ls-random'].reset_index()
ls_nn_df = df[df['name'] == 'ls-nn'].reset_index()
ils_df = df[df['name'] == 'ils'].reset_index()
ea_df = df[df['name'] == 'ea'].reset_index()

_df.head(n=5)

,index,name,instance_name,primary_graph,secondary_graph,time_ns,primary_cost,secondary_cost,total_cost
0,100,nn,kroA100,15-87-21-93-69-52-78-17-23-37-35-98-83-9-71-20...,4-36-77-51-95-38-29-47-99-70-40-13-2-45-28-33-...,5452309,13228,18698,31926
1,101,nn,kroA100,28-45-2-42-13-70-40-99-47-51-77-95-4-36-32-75-...,64-65-3-96-55-79-30-88-41-7-91-0-62-5-48-89-9-...,4997223,12480,16404,28884
2,102,nn,kroA100,4-36-77-51-95-38-29-47-99-70-40-13-2-45-28-33-...,52-78-17-23-37-35-98-83-9-71-20-73-58-16-14-10...,3864575,14436,12789,27225
3,103,nn,kroA100,84-67-72-49-43-1-53-39-63-68-80-24-60-50-86-8-...,69-65-64-3-96-55-79-30-88-41-7-91-0-62-5-48-89...,3490109,11855,18696,30551
4,104,nn,kroA100,12-32-75-94-81-49-43-72-67-84-38-29-95-77-51-4...,59-61-76-22-97-90-44-31-10-14-16-73-20-58-71-9...,1857895,12651,16412,29063


In [6]:
def get_data(solver_df, instance):
    instance_df = solver_df[solver_df['instance_name'] == instance]
    instance_df = instance_df.reset_index()
    
    tc = instance_df['total_cost']
    best_cost = instance_df.iloc[tc.idxmin('column')]['total_cost']
    worst_cost = instance_df.iloc[tc.idxmax('column')]['total_cost']
    avg_cost = tc.mean()
    
    time = instance_df['time_ns']
    best_time = instance_df.iloc[time.idxmin('column')]['time_ns'] / 1000000
    worst_time = instance_df.iloc[time.idxmax('column')]['time_ns'] / 1000000
    avg_time = time.mean() / 1000000
    
    best_primary_graph = instance_df.iloc[tc.idxmin('column')]['primary_graph']
    best_secondary_graph = instance_df.iloc[tc.idxmin('column')]['secondary_graph']

    return {
        'best_result': {
            'primary': best_primary_graph,
            'secondary': best_secondary_graph
        },
        'best_cost': best_cost,
        'worst_cost': worst_cost,
        'avg_cost': avg_cost,
        'best_time': best_time,
        'worst_time': worst_time,
        'avg_time': avg_time
    }

In [8]:
get_data(ea_df, 'kroA150')

{'avg_cost': 34291.81,
 'avg_time': 6933.28595979,
 'best_cost': 30033,
 'best_result': {'primary': '78-136-87-15-21-93-69-52-133-105-89-48-5-62-0-129-91-7-41-121-79-30-88-132-137-147-141-104-66-107-57-27-92-130-46-108-90-97-22-44-31-10-14-16-140-58-73-20-71-112-9-83-35-37-23-17-103-101-110-98-126-18-74-96-142-55-138-118-3-64-65-128-25-123-117',
  'secondary': '40-70-99-47-13-2-45-28-131-111-106-120-29-100-38-95-77-51-4-36-146-12-32-145-102-75-94-125-81-115-49-43-113-143-63-39-53-1-72-67-84-134-139-116-6-8-56-144-86-124-50-60-24-80-68-33-82-54-148-11-26-85-34-149-61-59-76-109-19-119-114-122-42-127-135'},
 'best_time': 5571.657713,
 'worst_cost': 37732,
 'worst_time': 14555.650318}

In [9]:
results = {
    'NN kroA100': get_data(nn_df, 'kroA100'),
    'NN kroA150': get_data(nn_df, 'kroA150'),
    'GC kroA100': get_data(gc_df, 'kroA100'),
    'GC kroA150': get_data(gc_df, 'kroA150'),
    'LS Random kroA100': get_data(ls_random_df, 'kroA100'),
    'LS Random kroA150': get_data(ls_random_df, 'kroA150'),
    'LS NN kroA100': get_data(ls_nn_df, 'kroA100'),
    'LS NN kroA150': get_data(ls_nn_df, 'kroA150'),
    'ILS kroA100': get_data(ils_df, 'kroA100'),
    'ILS kroA150': get_data(ils_df, 'kroA150'),
    'EA kroA100': get_data(ea_df, 'kroA100'),
    'EA kroA150': get_data(ea_df, 'kroA150')
}

In [10]:
COLUMNS_COST = ['Min Cost', 'Max Cost', 'Avg Cost']
COLUMNS_TIME = ['Min Time [ms]', 'Max Time [ms]', 'Avg Time [ms]']

def table(data_dict, key, columns):
    series = []
    indexes = []
    
    for solver, values in data_dict.items():
        best = values['best_{}'.format(key)]
        worst = values['worst_{}'.format(key)]
        avg = values['avg_{}'.format(key)]
        
        indexes.append(solver)
        series.append(pd.Series([best, worst, avg], index=columns))
    
    return pd.DataFrame(series, index=indexes)

def parse_vector(vector):
    results = vector.split('-')
    return " - ".join(results)

def vectors(data_dict):
    for solver, values in data_dict.items():
        print('Solver: {}'.format(solver))
        print(parse_vector(values['best_result']['primary']))
        print()
        print(parse_vector(values['best_result']['secondary']))
        print()

In [11]:
table(results, 'cost', COLUMNS_COST)

,Min Cost,Max Cost,Avg Cost
GC kroA100,28450.0,36401.0,31729.88
EA kroA100,25070.0,33286.0,29378.49
EA kroA150,30033.0,37732.0,34291.81
LS Random kroA100,43370.0,69950.0,54133.23
ILS kroA150,59896.0,61045.0,61033.51
LS Random kroA150,62194.0,85295.0,71767.55
GC kroA150,35039.0,43548.0,38458.19
LS NN kroA100,47754.0,47754.0,47754.00
NN kroA150,30664.0,38806.0,35038.48
ILS kroA100,42989.0,50411.0,47713.75


In [12]:
table(results, 'time', COLUMNS_TIME)

,Min Time [ms],Max Time [ms],Avg Time [ms]
GC kroA100,3.653104,115.813855,9.061932
EA kroA100,2604.269935,10319.899018,4130.705263
EA kroA150,5571.657713,14555.650318,6933.285960
LS Random kroA100,53.155598,280.841086,90.544357
ILS kroA150,27494.544112,44598.965537,32325.368920
LS Random kroA150,200.552580,471.029548,294.563146
GC kroA150,10.662524,12.045759,11.073706
LS NN kroA100,81.763279,141.734193,89.455478
NN kroA150,3.229817,4.356971,3.425506
ILS kroA100,7090.464706,11180.122152,7766.326834


In [13]:
vectors(results)

Solver: GC kroA100
52 - 78 - 87 - 15 - 21 - 93 - 69 - 18 - 74 - 96 - 3 - 64 - 65 - 25 - 89 - 48 - 5 - 62 - 9 - 83 - 17 - 23 - 37 - 35 - 98 - 71 - 20 - 73 - 58 - 10 - 16 - 14 - 31 - 46 - 0 - 91 - 7 - 55 - 79 - 30 - 41 - 88 - 66 - 27 - 92 - 57 - 90 - 97 - 22 - 44

95 - 77 - 51 - 4 - 36 - 38 - 29 - 47 - 13 - 99 - 70 - 40 - 2 - 28 - 45 - 42 - 33 - 82 - 54 - 84 - 67 - 49 - 72 - 43 - 81 - 94 - 12 - 32 - 75 - 1 - 63 - 39 - 53 - 68 - 80 - 24 - 60 - 50 - 86 - 8 - 6 - 56 - 11 - 26 - 85 - 19 - 34 - 61 - 76 - 59

Solver: EA kroA100
26 - 85 - 34 - 19 - 11 - 6 - 8 - 56 - 86 - 50 - 60 - 24 - 80 - 68 - 72 - 49 - 43 - 1 - 53 - 39 - 63 - 67 - 84 - 38 - 29 - 95 - 77 - 51 - 4 - 36 - 32 - 75 - 12 - 94 - 81 - 47 - 99 - 70 - 40 - 13 - 2 - 45 - 28 - 33 - 82 - 54 - 42 - 61 - 59 - 76

93 - 21 - 15 - 87 - 52 - 78 - 17 - 23 - 37 - 35 - 98 - 83 - 9 - 71 - 20 - 73 - 58 - 16 - 14 - 10 - 31 - 90 - 97 - 22 - 44 - 46 - 62 - 5 - 48 - 89 - 18 - 74 - 96 - 55 - 79 - 30 - 88 - 41 - 7 - 91 - 0 - 92 - 27 - 66 - 57 - 3 - 64 - 